# The spelled-out intro to language modeling: building makemore

Lecture: https://www.youtube.com/watch?v=PaCmpygFfXo

Program that attempts to makes "more" of whatever you give it. E.g. pass it names, create more things that sound like names.

Character-level language model that treats what you pass it as a sequence of characters; attempts to predict next in sequence.

Allows different approaches:

  * Bigram
  * Bag of words
  * MLP (multi-layer perceptron)
  * RNN (recurrent neural network, i.e. looping input)
  * GRU (gated recurrent unit ??)
  * Transformer neural network

Look at working set of ~32k names.

In [1]:
# Set working directory and load file
import os
os.chdir('../nn-zero-to-hero/makemore/')

words = open('names.txt', 'r').read().splitlines()

In [2]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
len(words)

32033

In [24]:
# Word lengths
print(
    "min = ", str(min(len(w) for w in words)), "\nmax = ", str(max(len(w) for w in words))
)

min =  2 
max =  15


Consider one example:

In [15]:
words[3]

'isabella'

Lots of information contained in the training example: `s` can come after `i`, `a` can come after `s`, ..., `l` can come after itself, `a` can be the end of a word. Dataset is tens of thousands of examples of this.

# Bigram approach

Look at local structure only. Use previous letter to predict the next one. Basic approach: look at sliding character pairs in word. Consider the first example, emma:

In [16]:
for w in words[:1]:
    for ch1, ch2 in zip(w, w[1:]):
        print(ch1, ch2)

e m
m m
m a


`zip(w, w[1:])` pairs the two iterators, offset - so `emma` gets lined up with `mma` and zip ends after 3 because `mma` runs out of letters. But there's more info not yet captured, like `e` coming first and `a` coming last.

Workaround: use a special character for start and end characters.

In [22]:
for w in words[:1]:
    chs = ['<S>'] + list(w) + ['<E>'] # New line: append <S>tart and <E>nd characters
    for ch1, ch2 in zip(chs, chs[1:]):
        print(ch1, ch2)

<S> e
e m
m m
m a
a <E>


In [23]:
for w in words[:3]: # Let's look at a few more names
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        print(ch1, ch2)

<S> e
e m
m m
m a
a <E>
<S> o
o l
l i
i v
v i
i a
a <E>
<S> a
a v
v a
a <E>


This is the basic data format. Next step is to store counts, for example in a dictionary `b`.

In [26]:
b = {}

for w in words[:3]:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2) # dictionary key
        b[bigram] = b.get(bigram, 0) + 1 # access or create dictionary key; increment count by 1 (intialize to 0)
        # print(ch1, ch2)

b

{('<S>', 'e'): 1,
 ('e', 'm'): 1,
 ('m', 'm'): 1,
 ('m', 'a'): 1,
 ('a', '<E>'): 3,
 ('<S>', 'o'): 1,
 ('o', 'l'): 1,
 ('l', 'i'): 1,
 ('i', 'v'): 1,
 ('v', 'i'): 1,
 ('i', 'a'): 1,
 ('<S>', 'a'): 1,
 ('a', 'v'): 1,
 ('v', 'a'): 1}

`a` is the ending character 3 times; looks right. Now we can get rid of the printing and look at the whole dataset.

In [27]:
b = {}

for w in words: # entire dataset
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

In [33]:
sorted( # return tuples sorted by frequency
    b.items() # returns (key, value) tuples
    , key = lambda kv: -kv[1] # forces sort by the 1st indexed item in the tuple, i.e. the value; negating fixes sort order
)[:20]

[(('n', '<E>'), 6763),
 (('a', '<E>'), 6640),
 (('a', 'n'), 5438),
 (('<S>', 'a'), 4410),
 (('e', '<E>'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('<S>', 'k'), 2963),
 (('l', 'e'), 2921),
 (('e', 'n'), 2675),
 (('l', 'a'), 2623),
 (('m', 'a'), 2590),
 (('<S>', 'm'), 2538),
 (('a', 'l'), 2528),
 (('i', '<E>'), 2489),
 (('l', 'i'), 2480),
 (('i', 'a'), 2445),
 (('<S>', 'j'), 2422)]

For efficiency, use PyTorch.